In [1]:
import numpy as np
import time
import ipyleaflet 
import geocoder
from ipyleaflet import Map, FullScreenControl, Icon, Marker, AwesomeIcon, WidgetControl, Rectangle
import geopy
from geopy import distance
import ipywidgets as widgets
import threading
import pyautogui
from main_otherfreq import Eyetracker


class Karta():
    def __init__(self, place,basezoom,nrofenemies,nrofneutrals):
        self.location = geocoder.osm(place)
        self.homelatlng=[self.location.lat, self.location.lng] #get latitude and longitude
        #-------Keep track of zoom level-----------
        self.basezoom=basezoom
        self.zoomlvl=basezoom
        self.bound_extra=0.1
        #------------------------------------------
        self.nrofenemies=nrofenemies
        self.nrofneutrals=nrofneutrals
        self.m=Map(zoom=basezoom,dragging=False,scroll_wheel_zoom=True,box_zoom=True, center=self.homelatlng) #initialize map
        self.objects= np.zeros((2,1))
        #-------Button for removing select icon-----------
        self.button1 = widgets.Button(description="Remove marker")
        self.button1.on_click(self.on_button_clicked1)
        #-------Eyeicon-marker for visualising clicks-----------
        self.eyeicon = AwesomeIcon( name='eye',  marker_color='black', icon_color='red', spin=False)
        #-------Selection-marker-----------
        self.selecticon = AwesomeIcon( name='check',  marker_color='black', icon_color='yellow', spin=False)
        self.selectmarker=Marker(icon=self.selecticon ,draggable=False,keyboard=True,title="Selected", location=[])
        self.markerout=False
        self.zoomed=False
        
        
        self.centroid_icon = AwesomeIcon( name='eye',  marker_color='black', icon_color='black', spin=False)
        self.centroid_marker=Marker(icon=self.centroid_icon ,draggable=False,keyboard=True,title="", location=[])
        self.rectangle = Rectangle(bounds=((),()))
        
        self.latscale=(360/40075)
        self.longscale=40075 
        
        self.bounds_old=None
        
        self.lat=[]
        self.lng=[]
        self.count=0
        
    def iniate_map_with_home(self):
        self.m.add_control(FullScreenControl()) # Make fullscreen available 
        home_ship = AwesomeIcon( name='ship',  marker_color='blue', icon_color='black', spin=False) #Icon for home
        home= Marker(icon=home_ship ,draggable=False, location=(self.homelatlng)) #marker for home
        self.objects[0][0]=self.homelatlng[0] #Adding home coordinates to object list
        self.objects[1][0]=self.homelatlng[1]
        self.m.add_layer(home) #add home marker
        display(self.m)
        
        
        
    def add_specific_enemies(self,enemies):
        
        
        self.lat.append(enemies[0])
        self.lng.append(enemies[1])
        
        loc=(enemies[0],enemies[1])
        self.m.add_layer(Marker(icon=self.selecticon ,draggable=False,keyboard=True,title="", location=loc))
        if self.count>=10:
            print("latitudes",self.lat)
            print("-------------------")
            print("longitudes",self.lng)
        self.count+=1 
        
        
            #latituderange=np.random.uniform(low=54, high=70, size=(1,self.nrofenemies)) #Init coordinates
            #longituderange=np.random.uniform(low=2, high=31, size=(1,self.nrofenemies))    
            #enemy_coordinates=[latituderange[0],longituderange[0]]
            #self.objects=np.concatenate([self.objects,enemy_coordinates], axis=1 )
            #icon = AwesomeIcon( name='fighter-jet',  marker_color='red', icon_color='black', spin=False)
            #for i in range(nrofenemies):
            #    enemy=Marker(icon=icon ,draggable=False,rise_on_hover=True,location=([latituderange[0][i],longituderange[0][i]]))
            #    self.m.add_layer(enemy)
            #      
   
            

    #def add_specific_enemy(self):
    #    enemy=[[59.369172,59.254290],[18.334235,18.651818]]
    #    self.objects=np.concatenate([self.objects,enemy], axis=1 )
    #    icon = AwesomeIcon( name='fighter-jet',  marker_color='red', icon_color='black', spin=False)
    #    for i in range(2):
    #        specificenemy=Marker(icon=icon ,draggable=False, location=[enemy[0][i],enemy[1][i]])
    #        self.m.add_layer(specificenemy)
        
        
    def shortest_distance(self, point):
        threshold=200 #Threshold can be seen as Eyetracker sensitivity, set heuristically as of now.
        distances=np.array([geopy.distance.distance( (self.objects[0][i],self.objects[1][i]),(point[0],point[1]) ).km for i in range(np.size(self.objects,1))])
        idx=np.where(distances<=threshold)
        print("Start zoom innan ", self.m.zoom)
        
        print("ONE CLICK")
        if len(idx[0])>1:
            if self.zoomed==False:
                #print("self.m.zoom",self.m.zoom)
                #print("self.m.bounds", self.m.bounds)
                #self.bounds_old=self.m.bounds
                
                print("Zoom needed!")
                #max_idx=np.where(distances==max(distances[idx]))
                #max_dist=distances[max_idx]
                centroid=list(np.mean(self.objects[:,idx[0]], axis=1))
                centroid_arr=np.array([ [ centroid[0] ], [centroid[1]] ])
                diff=abs(np.subtract(centroid_arr,self.objects[:,idx[0]]))
               
                lat_dist=max(diff[0])
                lng_dist=max(diff[1])
                #print("lat_dist",lat_dist)
                #print("lng_dist",lng_dist)
                #if lat_dist>=lng_dist:
                #    print("Första")
                #    lng_dist=2*lng_dist
                #if lng_dist>=lat_dist:
                #    print("andra")
                #    lat_dist=2*lat_dist
                
                
                #distance_centroid=np.array([geopy.distance.distance( (self.objects[0][i],self.objects[1][i]),(centroid[0],centroid[1]) ).km for i in idx[0]])
                #max_dist=np.max(distance_centroid)
                #lat_degree=max_dist*self.latscale
                #
                #lng_scale=np.cos(centroid[0])*self.longscale
                #lng_degree=max_dist*(360/lng_scale)
                #
                #test=(90/10001.965729)*max_dist
                
                
                
                bound_tuple=(( (centroid[0]-lat_dist, centroid[1]-lng_dist ),( centroid[0]+lat_dist,centroid[1]+lng_dist)  ))
                
                self.m.fit_bounds(bound_tuple)
                
                self.rectangle.bounds=bound_tuple
                self.centroid_marker.location=centroid
                self.m.add_layer(self.centroid_marker)
                self.m.add_layer(self.rectangle)
                
                
                
                self.zoomed=True
                
            elif self.zoomed==True:
                
                print("We are zoomed in and can now select!")
                if self.markerout==True:
                    self.m.remove_layer(self.selectmarker)
                    self.markerout=False
                    
              
                closest_idx=np.where(distances==min(distances[idx]))[0][0]
                closest_latlng=[self.objects[0][closest_idx],self.objects[1][closest_idx]]
                self.selectmarker.location=closest_latlng
                self.m.add_layer(self.selectmarker)
            
                self.markerout=True
                self.m.zoom=basezoom
                self.m.remove_layer(self.rectangle)
                self.m.remove_layer(self.centroid_marker)
                self.zoomed=False
                
                #print("self.bounds_old",self.bounds_old)
                #print("self.m.bounds", self.m.bounds)
                #print("self.m.zoom",self.m.zoom)
                
                
            
            
            
 
        if len(idx[0])==1: #Only one within threshold.
            print("We can select without zoom!")
            if self.markerout==True:
                self.m.remove_layer(self.selectmarker)
                self.markerout=False
           
            closest_latlng=[self.objects[0][idx[0][0]],self.objects[1][idx[0][0]]]
            self.selectmarker.location=closest_latlng
            self.m.add_layer(self.selectmarker)
            self.markerout=True
            self.zoomed=False
            self.m.fit_bounds(self.bounds_old)
            
            self.m.remove_layer(self.rectangle)
            self.m.remove_layer(self.centroid_marker)
            self.bounds_old=None
            
        
        print("Start zoom efter ", self.m.zoom)
        print("Slut av funktion!")
        print("----------------------")
        
        
    def on_button_clicked1(self,*args):
        self.markerout=False
        self.m.remove_layer(self.selectmarker)
        self.m.remove_layer(self.centroid_marker)
        self.m.remove_layer(self.rectangle)
        
 
    
    def get_map_click(self,**kwargs):
    
        
        if kwargs.get('type')=='preclick':#kwargs.get('type') == 'click' or 
            
            
            locationclick=kwargs.get('coordinates')
            self.add_specific_enemies(locationclick)
          

        
        
        
        
    




if __name__ == "__main__":
    
   
    #-------- Variables for map initiation -----------
    place='Stockholm' #Map center/ Home object location, in our case STHLM
    basezoom=5 #First zoom appearence
    nrofneutrals=10 #Number of neutral objects on map
    nrofenemies=3 #Number of enemies displayed on map
        
    #--------------- Map initialization -----------
    map_sthlm=Karta(place,basezoom,nrofenemies,nrofneutrals)
    map_sthlm.iniate_map_with_home()
    
    
    
    
    #------------------Draw data on map----------------------------

    remove_select = WidgetControl(widget=map_sthlm.button1, position='bottomleft')
    map_sthlm.m.add_control(remove_select)
    map_sthlm.m.on_interaction(map_sthlm.get_map_click)
    
     #------------------Eyetracker init, should handle multiple executions----------------------------
    
    #start Eyetracker 
    #et=Eyetracker()
    #et.start_eyetracker()
    #tread.start()
    #tread = threading.Thread(target=et.start_eyetracker())
    #tread.start()
    


    






Map(center=[59.3251172, 18.0710935], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…